# Load packages

In [75]:
#%reset

from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy import fftpack
import pandas as pd
from pathlib import Path
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from ephyviewer import mkQApp, MainViewer, TraceViewer
from ephyviewer import AnalogSignalSourceWithScatter
import ephyviewer
from scipy.stats import zscore
from scipy.interpolate import interp1d
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
import ephyviewer
import json
import matplotlib.cm as cm
%matplotlib widget

from scipy.signal import find_peaks
from scipy.signal import chirp, find_peaks, peak_widths

# Choose OpenEphys folder (to modify)

In [ ]:
dpath = "//10.69./.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"
try:
    %store -r dpath
except:
    print("data not in strore")
    dpath ="//10.69./.1/crnldata/waking///L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Select recording folder from OpenEphys</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

# stop when "continuous" & "events" folders appears

# Load data

In [ ]:
folder = Path(dpath)

# Load metadata 
filepath = os.path.join(folder, f'structure.oebin')
with open(filepath) as f:
    metadata = json.load(f)
samplerate=metadata['continuous'][0]['sample_rate']  
print('sample rate = ', samplerate, 'Hz')
numchannel=metadata['continuous'][0]['num_channels']  
print(numchannel, 'channels recorded')

# Load TTLs
for file_path in folder.glob('**/TTL/timeStamps.npy'):
    print('TTL opto file = ', file_path)
    TTL_Opto_o = np.load(file_path)
    TTL_Opto_duration =[round(TTL_Opto_o[i+1] - TTL_Opto_o[i],3) for i in range(len(TTL_Opto_o) - 1)[::2]]
    TTL_Opto= TTL_Opto_o[::2] # remove the TTL for laser OFF, only keep TTL for laser ON. CAUTION /!\ works only if it started with a TTL for laser ON

    print(TTL_Opto.shape[0], 'opto stimulations')

# Load LFPs 
for file_path in folder.glob('**/continuous.dat'):
    print('LFPs data file = ', file_path)
    LFPs = np.fromfile(file_path, dtype="int16")
    All = LFPs.reshape(-1,numchannel)
    LFPs2 = np.fromfile(file_path, dtype="uint16")
    All2 = LFPs2.reshape(-1,numchannel)
    
# Load LFPs timestamps 
for file_path in folder.glob('**/continuous/*/timeStamps.npy'):
    print('LFPs timestamps file = ', file_path)
    LFPtimestamps = np.load(file_path)  

print(round(All.shape[0]/samplerate/60), 'min of recording')

# Select LFPs channels (to modify)

In [ ]:
Channels = '//10.69.168.1/crnldata/waking/audrey_hay/AurelieB/LFPChannels_perMice.xlsx'  # TO MODIFY
mice=folder.parts[-1].split('_')[0] # TO MODIFY
print(mice)

allchannels = pd.read_excel(Channels)

PFCch1=int(allchannels[mice][0].split(',')[0])
PFCch2=int(allchannels[mice][0].split(',')[1])
CA1ch1=int(allchannels[mice][2].split(',')[0])
CA1ch2=int(allchannels[mice][2].split(',')[1])
S1ch1=int(allchannels[mice][1].split(',')[0])
S1ch2=int(allchannels[mice][1].split(',')[1])
EMGch1=int(allchannels[mice][3])

PFC  =  All[:, PFCch1]-All[:, PFCch2] 
CA1  =  All[:, CA1ch1]-All[:, CA1ch2] 
S1  =  All[:, S1ch1]-All[:, S1ch2] 
EMG  =  All2[:, EMGch1]


### Synchronized TTLs to LFPs start (NOT NEEDED)

In [ ]:
"""A = []
#for file_path in folderMouse.glob('**/*sync_messages.txt'):
file_path= f'{folder}/sync_messages.txt'
with open(file_path, "r") as f:
    lines = f.readlines()
# Loop through all lines, ignoring header.
# take last element to list (i.e. the process name)
    for l in lines[1:]:
        A.append(l.split()[-1])

# remove acquisition frequency that is normally always 25 kHz
initial_OE_start = int(' '.join([x.split('@')[0] for x in A]))
print("Initial start of OE", round(initial_OE_start/samplerate,2), 'sec')

TTL_Opto_n = (TTL_Opto - initial_OE_start/samplerate)
print('Synchronized TTL = ', TTL_Opto_n)
"""

# Get response of each LFPs channels

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# If plt.xlim was overwritten, reset it using ax
AllEPSPs = []
for ttl in TTL_Opto:   
    idx = (np.abs(LFPtimestamps - ttl)).argmin()  # Find the closest LFP timestamps to the TTL
    AllEPSPs.append(All[idx-round(0.5*samplerate):idx+round(0.5*samplerate), :])  # 500 ms before and after TTL

plt.close()
fig, ax = plt.subplots(figsize=(10, 4))  # Use axes object 'ax'

# Create time axis for plotting
time_axis = np.linspace(-0.5, 0.5, np.shape(AllEPSPs)[1]) 

# Calculate the mean across AllEPSPs
mEPSPs = np.mean(AllEPSPs, axis=0)

# Generate colors for the channels
colors = cm.rainbow(np.linspace(0, 1, np.shape(mEPSPs)[1]))

# Plot each channel's data
for i in np.arange(0, np.shape(AllEPSPs)[2]):
    ax.plot(time_axis, mEPSPs[:, i], label=f'Channel #{i}', color=colors[i])

# Set labels and title
ax.set_xlabel("Time (s)")
ax.set_title('All opto stim')

# Set the x-axis and y-axis limits using ax methods
ax.set_xlim([-0.5, 0.5])  # Adjust x-axis limits
ax.set_ylim(np.min(mEPSPs), np.max(mEPSPs))  # Adjust y-axis based on the data range

# Add legend
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncols=2)

# Display the plot
plt.show()

# Adjust subplot spacing
plt.subplots_adjust(right=0.8)  # Adjust right side to give space for the legend


In [ ]:
AllEPSPs=[]
for ttl in TTL_Opto:   
    idx = (np.abs(LFPtimestamps - ttl)).argmin() # find the closest LFP timestamps to the TTL
    AllEPSPs.append(All[idx-round(0.5*samplerate):idx+round(0.5*samplerate), :]) #500 ms before and after TTL

plt.close()
plt.figure(figsize=(10, 4))
time_axis = np.linspace(-0.5, 0.5, np.shape(AllEPSPs)[1]) 
mEPSPs=np.mean(AllEPSPs, axis=0)
colors = cm.rainbow(np.linspace(0, 1, np.shape(mEPSPs)[1]))
for i in np.arange(0,np.shape(AllEPSPs)[2]):
    plt.plot(time_axis, mEPSPs[:,i], label=f'Channel #{i}', color=colors[i])
plt.xlabel("Time (s)")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncols=2)
plt.title('All opto stim')
plt.show()
plt.subplots_adjust(right=.5)  


# Get response for one LFP

In [1131]:
SelectedLFP=All[:,31] # to change for PFC or S1 or etc

In [ ]:
nb_differentStim=len(np.unique(TTL_Opto_duration)) # how many different opto stim are performed in the arduino protocol (here a loop of 19 stim with different durations)
DurStim=np.unique(TTL_Opto_duration) # durations of each unique stim (from 1 ms to 20 ms with a step of 1)

AllStim={}
for i, ttl  in enumerate(TTL_Opto):
    idx = (np.abs(LFPtimestamps - ttl)).argmin() # find the closest LFP timestamps to the TTL    
    if str(TTL_Opto_duration[i]) in AllStim:
        AllStim[str(TTL_Opto_duration[i]) ]=pd.concat([AllStim[str(TTL_Opto_duration[i]) ],pd.DataFrame(SelectedLFP[idx-round(0.5*samplerate):idx+round(0.5*samplerate)])],axis=1)
    else:
        AllStim[str(TTL_Opto_duration[i]) ]=pd.DataFrame(SelectedLFP[idx-round(0.5*samplerate):idx+round(0.5*samplerate)])
   
AllStim_sorted = dict(sorted(AllStim.items()))
meanAllStim= pd.DataFrame([np.mean(AllStim_sorted[key], axis=1) for key in AllStim_sorted.keys()]).T
meanAllStim.columns=DurStim*1000

Amplitude=meanAllStim[round(0.5*samplerate):round(0.6*samplerate)].max()-meanAllStim[round(0.5*samplerate):round(0.6*samplerate)].min()
Max=meanAllStim[round(0.5*samplerate):round(0.6*samplerate)].max()
HalfMaxWidth = [((np.abs(meanAllStim[i][meanAllStim[i][meanAllStim[i]==Max[i]].index[0]:] - Max[i]/2)).argmin() -round(0.5*samplerate) + meanAllStim[i][meanAllStim[i]==Max[i]].index[0])/samplerate*1000 for i in meanAllStim.columns]


# Plot
plt.close()
fig, axs = plt.subplots(2,2, figsize=(12,6))
fig.suptitle(f'Mice = {mice}, LFP = ', fontsize=15) # Change to indicate where is located that LFP

time_axis = np.linspace(-0.5, 0.5, meanAllStim.shape[0]) 
grey_shades = np.linspace(0.9, 0.1, len(meanAllStim.columns)) 
for i, col in enumerate(meanAllStim.columns):
    axs[0,0].plot(time_axis, meanAllStim[col], color=(grey_shades[i], grey_shades[i], grey_shades[i]), label=f'{col} ms')
axs[0,0].plot(time_axis, np.mean(meanAllStim, axis= 1), color='red')
axs[0,0].set(xlabel="Time (s)", ylabel="Averaged EPSPs")
axs[0,0].set(xlim=(-0.1, .4))
axs[0,0].legend(loc='upper center', bbox_to_anchor=(2.5, .9),
          fancybox=True, shadow=True, ncol=1)

axs[0,1].plot(DurStim*1000,Amplitude, 'k')
axs[0,1].set(xlabel="Opto stim duration (ms)", ylabel="Amplitude ")

axs[1,0].plot(DurStim*1000,Max, 'k')
axs[1,0].set(xlabel="Opto stim duration (ms)", ylabel="Maximum ")

axs[1,1].plot(DurStim*1000, HalfMaxWidth, 'k')
axs[1,1].set(xlabel="Opto stim duration (ms)", ylabel="HalfWidth (ms)")

plt.subplots_adjust(right=.8)  
plt.show()

